In [18]:
import asyncio
from websockets.server import serve, WebSocketServerProtocol
from websockets.client import connect, WebSocketClientProtocol
import functools

In [19]:
"""
A function to debug URP blocks over a websocket proxy

:param prefix: A string denoting the URP message prefix (normally either 'urp ' or 'urp: ')
               remember to include any trailing spaces
:yields: A list with any messages that should be forwarded to the destination.
         You must .send() the next byte of data or call next(). If you call next
         it is garuanteed to be a no-op, however you can catch a StopIteration if
         the generator has finished, which allows you to directly read into send
         without needing to store the send in-case the generator has finished. The
         generator will never finish after recieving a plain next(). If you call next
         the messages that were yielded *will be yielded again the next time you call
         next or send*
:returns: A list with any messages that should be forwarded to the destination
:raises: IOError if a message is given that is invalid URP (e.g. too long)

- This function prints debug information on stdout.
- This function will not necessarily pass through messages directly, however the messages
  it asks you to forward will always be valid URP with an identical meaning to before
"""
def debug_block(prefix):
    print("[SIDECAR] waiting for a new block")
    block_started = False
    messages_left_in_block = 0
    expected_bytes = 8

    forward = []

    while messages_left_in_block or not block_started:
        message = prefix.encode()
        while len(message) < expected_bytes + len(prefix):
            next_part = yield forward
            if next_part is None: continue
            forward = []
            if not next_part.startswith(prefix.encode()):
                forward.append(next_part)
                continue
            message += next_part[len(prefix):]

        if len(message) != expected_bytes + len(prefix):
            print(f"Message length ({len(message)}) was longer than expected ({expected_bytes})")
            print("Message:", message)
            raise IOError("Message was too long")

        unprefixed = message[len(prefix):]

        if messages_left_in_block == 0:
            # Block header message
            expected_bytes = int.from_bytes(unprefixed[0:4])
            messages_left_in_block = int.from_bytes(unprefixed[4:])
            print(f"[SIDECAR] incoming block of {messages_left_in_block} messages ({expected_bytes} bytes)")
            block_started = True

            assert messages_left_in_block == 1 # We cannot yet handle multi-message blocks
        else:
            messages_left_in_block -= 1
            print(f"[SIDECAR] Got a message ({len(unprefixed)} bytes)")

        if messages_left_in_block == 0:
            expected_bytes = 8

        forward.append(message)

    print("[SIDECAR] Finished processing block")
    return forward


In [20]:
async def from_sidecar(sidecar: WebSocketServerProtocol, libreoffice: WebSocketClientProtocol):
    while sidecar.open:
        try:
            gen = debug_block("urp ")
            while True:
                next(gen)
                for message in gen.send(await sidecar.recv()):
                    await libreoffice.send(message)
        except StopIteration as e:
            for message in e.value:
                await libreoffice.send(message)

async def from_libreoffice(sidecar: WebSocketServerProtocol, libreoffice: WebSocketClientProtocol):
    async for message in libreoffice:
        if type(message) == str:
            # Text message
            print(f"[LO({len(message)} chars)]: {message}")
        else:
            # Binary message
            print(f"[LO({len(message)} bytes)]: {message}")

        await sidecar.send(message)

async def sidecar(sidecar: WebSocketServerProtocol):
    print(f"Got a client on {sidecar.path}")
    async with connect(f"ws://localhost:9980{sidecar.path}") as libreoffice:
        await asyncio.gather(
            from_sidecar(sidecar, libreoffice),
            from_libreoffice(sidecar, libreoffice),
        )

In [21]:
async with serve(sidecar, "localhost", 9981) as sidecar:
    await sidecar.start_serving()
    while sidecar.is_serving():
        await asyncio.sleep(0)

Got a client on /cool/http%3A%2F%2Flocalhost%3A9980%2Fwopi%2Ffiles%2Fhome%2Fskyler%2FCode%2Fcollabora%2Fonline%2Ftest%2Fdata%2Fhello-world.odt%3Faccess_token%3Dtest%26access_token_ttl%3D0/ws
[SIDECAR] waiting for a new block
[LO(21 chars)]: statusindicator: find
[LO(24 chars)]: statusindicator: connect
[LO(22 chars)]: statusindicator: ready
[LO(10 chars)]: perm: edit
[LO(49 chars)]: filemode:{"readOnly": false, "editComment": true}
[LO(537 chars)]: wopi: {"BaseFileName":"hello-world.odt","DisableCopy":false,"DisableExport":false,"DisableInactiveMessages":false,"DisablePrint":false,"DownloadAsPostMessage":false,"EnableInsertRemoteImage":false,"EnableRemoteLinkPicker":false,"EnableShare":false,"FileUrl":"","HideExportOption":false,"HidePrintOption":false,"HideRepairOption":false,"HideSaveOption":false,"HideUserList":"false","IsOwner":false,"PostMessageOrigin":"http://localhost:9980","SupportsRename":false,"UserCanNotWriteRelative":true,"UserCanRename":false,"UserCanWrite":true}
[LO(40 ch

connection handler failed
Traceback (most recent call last):
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 959, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1161, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/framing.py", line 68, in read
    data = awai

Got a client on /cool/http%3A%2F%2Flocalhost%3A9980%2Fwopi%2Ffiles%2Fhome%2Fskyler%2FCode%2Fcollabora%2Fonline%2Ftest%2Fdata%2Fhello-world.odt%3Faccess_token%3Dtest%26access_token_ttl%3D0/ws
[SIDECAR] waiting for a new block
[LO(21 chars)]: statusindicator: find
[LO(24 chars)]: statusindicator: connect
[LO(22 chars)]: statusindicator: ready
[LO(10 chars)]: perm: edit
[LO(49 chars)]: filemode:{"readOnly": false, "editComment": true}
[LO(537 chars)]: wopi: {"BaseFileName":"hello-world.odt","DisableCopy":false,"DisableExport":false,"DisableInactiveMessages":false,"DisablePrint":false,"DownloadAsPostMessage":false,"EnableInsertRemoteImage":false,"EnableRemoteLinkPicker":false,"EnableShare":false,"FileUrl":"","HideExportOption":false,"HidePrintOption":false,"HideRepairOption":false,"HideSaveOption":false,"HideUserList":"false","IsOwner":false,"PostMessageOrigin":"http://localhost:9980","SupportsRename":false,"UserCanNotWriteRelative":true,"UserCanRename":false,"UserCanWrite":true}
[LO(40 ch

connection handler failed
Traceback (most recent call last):
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 959, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1161, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/framing.py", line 68, in read
    data = awai

Got a client on /cool/http%3A%2F%2Flocalhost%3A9980%2Fwopi%2Ffiles%2Fhome%2Fskyler%2FCode%2Fcollabora%2Fonline%2Ftest%2Fdata%2Fhello-world.odt%3Faccess_token%3Dtest%26access_token_ttl%3D0/ws
[SIDECAR] waiting for a new block
[LO(21 chars)]: statusindicator: find
[LO(24 chars)]: statusindicator: connect
[LO(22 chars)]: statusindicator: ready
[LO(10 chars)]: perm: edit
[LO(49 chars)]: filemode:{"readOnly": false, "editComment": true}
[LO(537 chars)]: wopi: {"BaseFileName":"hello-world.odt","DisableCopy":false,"DisableExport":false,"DisableInactiveMessages":false,"DisablePrint":false,"DownloadAsPostMessage":false,"EnableInsertRemoteImage":false,"EnableRemoteLinkPicker":false,"EnableShare":false,"FileUrl":"","HideExportOption":false,"HidePrintOption":false,"HideRepairOption":false,"HideSaveOption":false,"HideUserList":"false","IsOwner":false,"PostMessageOrigin":"http://localhost:9980","SupportsRename":false,"UserCanNotWriteRelative":true,"UserCanRename":false,"UserCanWrite":true}
[LO(40 ch

...

connection handler failed
Traceback (most recent call last):
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 959, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1161, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/framing.py", line 68, in read
    data = awai

Got a client on /cool/http%3A%2F%2Flocalhost%3A9980%2Fwopi%2Ffiles%2Fhome%2Fskyler%2FCode%2Fcollabora%2Fonline%2Ftest%2Fdata%2Fhello-world.odt%3Faccess_token%3Dtest%26access_token_ttl%3D0/ws
[SIDECAR] waiting for a new block
[LO(21 chars)]: statusindicator: find
[LO(24 chars)]: statusindicator: connect
[LO(22 chars)]: statusindicator: ready
[LO(10 chars)]: perm: edit
[LO(49 chars)]: filemode:{"readOnly": false, "editComment": true}
[LO(537 chars)]: wopi: {"BaseFileName":"hello-world.odt","DisableCopy":false,"DisableExport":false,"DisableInactiveMessages":false,"DisablePrint":false,"DownloadAsPostMessage":false,"EnableInsertRemoteImage":false,"EnableRemoteLinkPicker":false,"EnableShare":false,"FileUrl":"","HideExportOption":false,"HidePrintOption":false,"HideRepairOption":false,"HideSaveOption":false,"HideUserList":"false","IsOwner":false,"PostMessageOrigin":"http://localhost:9980","SupportsRename":false,"UserCanNotWriteRelative":true,"UserCanRename":false,"UserCanWrite":true}
[LO(40 ch

connection handler failed
Traceback (most recent call last):
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 959, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1161, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/framing.py", line 68, in read
    data = awai

Got a client on /cool/http%3A%2F%2Flocalhost%3A9980%2Fwopi%2Ffiles%2Fhome%2Fskyler%2FCode%2Fcollabora%2Fonline%2Ftest%2Fdata%2Fhello-world.odt%3Faccess_token%3Dtest%26access_token_ttl%3D0/ws
[SIDECAR] waiting for a new block
[LO(21 chars)]: statusindicator: find
[LO(24 chars)]: statusindicator: connect
[LO(22 chars)]: statusindicator: ready
[LO(10 chars)]: perm: edit
[LO(49 chars)]: filemode:{"readOnly": false, "editComment": true}
[LO(537 chars)]: wopi: {"BaseFileName":"hello-world.odt","DisableCopy":false,"DisableExport":false,"DisableInactiveMessages":false,"DisablePrint":false,"DownloadAsPostMessage":false,"EnableInsertRemoteImage":false,"EnableRemoteLinkPicker":false,"EnableShare":false,"FileUrl":"","HideExportOption":false,"HidePrintOption":false,"HideRepairOption":false,"HideSaveOption":false,"HideUserList":"false","IsOwner":false,"PostMessageOrigin":"http://localhost:9980","SupportsRename":false,"UserCanNotWriteRelative":true,"UserCanRename":false,"UserCanWrite":true}
[LO(40 ch

connection handler failed
Traceback (most recent call last):
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 959, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1161, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/skyler/.local/lib/python3.11/site-packages/websockets/legacy/framing.py", line 68, in read
    data = awai